## 0. Создать папку source, поместить туда файлы из приложения ( имитируем папки с источником данных)

In [1]:
import os
import shutil 
import numpy as np
import pandas as pd
from datetime import datetime
import xlrd
import logging

## 1. Переместить файлы из папки source в папку data (имитируем перемещение)

In [2]:
#проверям на существование директорию
try:
    os.mkdir('data')
except:
    print('this folder already exist')
#вроверяем на существование 
try:
    shutil.move(r'source\Задача 6 CAPACITIES.xlsx',r'data\Задача 6 CAPACITIES.xlsx')
    shutil.move(r'source\Задача 6 REUTERS.xls',r'data\Задача 6 REUTERS.xls')
except:
    print('this files already exist or moved')

## 2. Прочесть подряд все excel файлы в директории data в dataframe, писать имя читаемого файла в debug

In [3]:
#read all excel file and put their name in debug.txt
directory = 'data\\'
all_files = os.listdir(directory)
all_excell_files = []
for i in all_files:
    if i.endswith('.xls') or i.endswith('.xlsx'):
        all_excell_files.append(i)
debug_file = open('debug.txt','w')
for index in all_excell_files:
    debug_file.write(index + '\n')
debug_file.close()

этот блок кода я оставил для своих нужд, как конспект 
for i in all_excell_files:
    exec('df%s = pd.read_excel(directory + str(i))' % all_excell_files.index(i))
    

In [4]:
#read files
df1 = pd.read_excel(r'data\Задача 6 REUTERS.xls', sheet_name=1)
df2 = pd.read_excel(r'data\Задача 6 CAPACITIES.xlsx')

In [5]:
df1.head()

,Flow ID,Vessel,Vessel IMO,Load Country,Load Port,Departure Date,Discharge Country,Discharge Port,Arrival Date,Product,...,Benchmark,Closing Date,Validity Date,Contract Basis,Terms,Import/Export,Status,Source Grade,Country of Origin,Last Updated
0,4060929,GASCHEM ORCA,9743916,United States,Houston,2020-02-02 13:27:51,United Kingdom,Teesport,2020-02-16 08:00:00,Liquefied Petroleum Gas,...,NaN,NaN,NaN,NaN,NaN,NaN,Vessel Underway,NaN,NaN,2020-02-02 17:36:04
1,4071408,ARCTIC GAS,9745512,United States,Philadelphia,2020-02-01 22:28:35,Morocco,Jorf Lasfar,2020-02-10 23:00:00,Liquefied Petroleum Gas,...,NaN,NaN,NaN,NaN,NaN,NaN,Vessel Underway,NaN,NaN,2020-02-02 23:36:34
2,4023622,BW LOYALTY,9350290,United States,Houston,2020-02-01 13:10:43,Belgium,Antwerp,2020-02-15 23:00:00,Liquefied Petroleum Gas,...,NaN,NaN,NaN,NaN,NaN,NaN,Vessel Underway,NaN,NaN,2020-02-01 23:36:44
3,4088223,SEA SPEED,9763045,United States,Philadelphia,2020-01-31 12:05:11,Netherlands,Vlissingen,2020-02-11 22:00:00,Liquefied Petroleum Gas,...,NaN,NaN,NaN,NaN,NaN,NaN,Vessel Underway,NaN,NaN,2020-02-02 05:35:57
4,4024822,GASCHEM BELUGA,9743928,United States,Houston,2020-01-22 16:17:14,United Kingdom,Teesport,2020-02-05 20:00:00,Liquefied Petroleum Gas,...,NaN,NaN,NaN,NaN,NaN,NaN,Vessel Underway,NaN,NaN,2020-01-22 23:36:09


In [6]:
df2.head()

,Commodity,Cause,Region,Country / Territory,Company,Site,Plant No.,Estimated Start Date,Expected Annual Capacity Change (kt),Total Annual Capacity (kt),Swing Capable
0,ACETIC ACID,Expected Capacity,NORTH AMERICA,UNITED STATES,CELANESE,CLEAR LAKE/TX,2,нояб. 2020,500000,200000,NaN
1,ACETIC ACID,Expected Capacity,NORTHEAST ASIA,CHINA,SHANGHAI HUAYI CHEMICAL,QINZHOU,2,янв. 2021,500000,50000,NaN
2,BENZENE,Expected Capacity,FORMER USSR,RUSSIAN FEDERATION,TANECO,NIZHNEKAMSK,1,февр. 2020,60000,60000,NaN
3,BENZENE,Expected Capacity,NORTHEAST ASIA,CHINA,ZHEJIANG PETROCHEMICAL,ZHOUSHAN,1,февр. 2020,300000,300000,NaN
4,BENZENE,Expected Capacity,NORTHEAST ASIA,CHINA,ZHEJIANG PETROCHEMICAL,ZHOUSHAN,2,февр. 2020,150000,150000,NaN


<p>3. Если в файле нет одной из колонок:</p>
<ul>
<li>Commodity</li>
<li>Cause</li> 
<li>Region</li>
<li>Country / Territory</li>
<li>Company</li>
<li>Site</li>
<li>Plant No.</li>
<li>Estimated Start Date</li>
<li>Expected Annual Capacity Change (kt)</li>
<li>Total Annual Capacity (kt)</li>
<li>Swing Capable</li>
выдать сообщение статуса error с помощью библиотеки logging.<b> 
Сообщение должно быть достаточно информативным. (не просто "ахтунг, тут ошибка")</b>
<br/>
<p>4. Если колонки присутствуют переместить файл из директории data в директорию valid_data, записав имя перемещаемого файла в debug</p>
<br/>
<p>5. Если колонки отсутствовали переместить файл из директории data в директорию invalid_data, записав имя перемещаемого файла в debug</p>

In [7]:
column_tuple = ('Commodity', 'Cause', 'Region', 'Country / Territory','Company', 'Site', 'Plant No.', 'Estimated Start Date',
                'Expected Annual Capacity Change (kt)', 'Total Annual Capacity (kt)', 'Swing Capable')

In [8]:
logging.basicConfig(filename='log_file.log')

In [9]:
for i in column_tuple:
    if i not in df1.columns:
        logging.error('This is very informative error! There is no column, from column_list, \nin your file "Задача 6 REUTERS.xls"')
        try:
            os.mkdir('invalid_data')
        except:
            print('this folder already exist')
        shutil.move(r'data\Задача 6 REUTERS.xls', r'invalid_data\Задача 6 REUTERS.xls')
        break
    else:
        try:
            os.mkdir('valid_data')
        except:
            print('this folder already exist')
        shutil.move(r'data\Задача 6 REUTERS.xls', r'valid_data\Задача 6 REUTERS.xls')
        break

In [10]:
for i in column_tuple:
    if i not in df2.columns:
        logging.error('This is very informative error! There is no column, from column_list, \nin your file "Задача 6 CAPACITIES.xlsx".')
        try:
            os.mkdir('invalid_data')
        except:
            print('this folder already exist')
        shutil.move(r'data\Задача 6 CAPACITIES.xlsx', r'invalid_data\Задача 6 CAPACITIES.xlsx')
        break
    else:
        try:
            os.mkdir('valid_data')
        except:
            print('this folder already exist')
        shutil.move(r'data\Задача 6 CAPACITIES.xlsx', r'valid_data\Задача 6 CAPACITIES.xlsx')
        break

## 7. Записать весь лог logging в файл, поместить файл в директорию logs, имя файла должно быть log_YYYY_MM_DD_HH_MM_SS.log

In [12]:
#пытался остановить так
#os.rename('log_file.log', 'log_.log')

PermissionError: [WinError 32] The process cannot access the file because it is being used by another process: 'log_file.log' -> 'log_.log'